In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
from pipeline_setup import *
import time
from scipy.optimize import minimize

In [2]:
# Set file locations
advanced_settings_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTanjc08kc5vIN-icUMzMEGA9bJuDesLX8V_u2Ab6zSC4MOhLZ8Jrr18DL9o4ofKIrSq6FsJXhPWu3F/pub?gid=0&single=true&output=csv"

# Read in emissions from other sectors
sector_emissions_dict = set_sector_emissions_dict()
# Read in the advanced settings from the google sheet
params = read_advanced_settings(advanced_settings_url)
# Set the datablock
datablock_init = datablock_setup()

# Set the scenario parameters
params_baseline = set_baseline_scenario(params)

# Also add the baseline parameters to the datablock
datablock_init.update(params_baseline)

In [3]:
# Name the output parameters of the current calculator function 
# (ideally these would be set and returned by the calculator)
z1_name = "SSR weight"
z2_name = "SSR prot"
z3_name = "SSR fat"
z4_name = "SSR kcal"
z5_name = "emissions"
z6_name = "herd size"

z_names = [z1_name, z2_name, z3_name, z4_name, z5_name, z6_name]

# Name the params we are going to vary
names_x = [
    "ruminant",
    "pig_poultry",
    "fish_seafood",
    "dairy",
    "eggs",
    "fruit_veg",
    "pulses",
    "meat_alternatives",
    "dairy_alternatives",
    "waste",

    "foresting_pasture",
    "land_BECCS",
    "land_BECCS_pasture",
    "horticulture",
    "pulse_production",
    "lowland_peatland",
    "upland_peatland",
    "pasture_soil_carbon",
    "arable_soil_carbon",
    "mixed_farming",

    "silvopasture",
    "methane_inhibitor",
    "stock_density",
    "manure_management",
    "animal_breeding",
    "fossil_livestock",

    "agroforestry",
    "nitrogen",
    "vertical_farming",
    "fossil_arable",

    "waste_BECCS",
    # "overseas_BECCS",
    # "DACCS",
    # "biochar",           
           ]

# Set the bounds for the optimisation - to be the same as for the grid
# Make a 2d plot
x_mins = (
    -100, # "ruminant"
    -100, # "pig_poultry"
    -100, # "fish_seafood"
    -50, # "dairy"
    -100, # "eggs"
    -100, # "fruit_veg"
    -100, # "pulses"
    0, # "meat_alternatives"
    0, # "dairy_alternatives"
    0, # "waste"

    10, # "foresting_pasture"
    0, # "land_BECCS"
    0, # "land_BECCS_pasture"
    -100, # "horticulture"
    -100, # "pulse_production"
    0, # "lowland_peatland"
    0, # "upland_peatland"
    0, # "pasture_soil_carbon"
    0, # "arable_soil_carbon"
    0, # "mixed_farming"

    0, # "silvopasture"
    0, # "methane_inhibitor"
    -100, # "stock_density"
    0, # "manure_management"
    0, # "animal_breeding"
    0, # "fossil_livestock"

    0, # "agroforestry"
    0, # "nitrogen"
    0, # "vertical_farming"
    0, # "fossil_arable"
    
    0, # "waste_BECCS"  
    # 0, # "overseas_BECCS"
    # 0, # "DACCS"
    # 0, # "biochar
          )

x_maxs = (
    50, # "ruminant"
    100, # "pig_poultry"
    100, # "fish_seafood"
    100, # "dairy"
    100, # "eggs"
    500, # "fruit_veg"
    500, # "pulses"
    100, # "meat_alternatives"
    100, # "dairy_alternatives"
    100, # "waste"

    50, # "foresting_pasture"
    100, # "land_BECCS"
    100, # "land_BECCS_pasture"
    500, # "horticulture"
    500, # "pulse_production"
    100, # "lowland_peatland"
    100, # "upland_peatland"
    100, # "pasture_soil_carbon"
    100, # "arable_soil_carbon"
    25, # "mixed_farming"

    100, # "silvopasture"
    100, # "methane_inhibitor"
    50, # "stock_density"
    100, # "manure_management"
    100, # "animal_breeding"
    100, # "fossil_livestock"

    100, # "agroforestry"
    100, # "nitrogen"
    100, # "vertical_farming"
    100, # "fossil_arable"
    
    40, # "waste_BECCS"  
    # 100, # "overseas_BECCS"
    # 200, # "DACCS"
    # 10, # "biochar
          ) 

x_bounds = [(x_mins[i], x_maxs[i]) for i in range(len(names_x)) ] # todo make this nd

In [4]:
# Create an class with anobjective function suitable for giving to the scipy minimizer
# Including a cache to avoid recomputing the same values
class FFCObjectiveWithCache:
    """A class to compute the objective function and constraints for FFC optimization with caching.
    Parameters:
        names_x (list): List of parameter names to be optimized.
        datablock_init (dict): Initial data block containing fixed parameters.
        params_default (dict): Default parameters for the optimization.
        sector_emissions_dict (dict): Dictionary containing emissions data for different sectors.
        verbosity (int): Level of verbosity for output messages.
    """
    def __init__(self, names_x, datablock_init, params_default, verbosity=0):
        self.names_x = names_x
        self.datablock_init = datablock_init
        self.params_default = params_default
        self._cache = {}
        self.verbosity = verbosity

        # Define the names of the z variables returned by the calculator
        self.z_names = ["SSR weight",
                   "SSR prot",
                   "SSR fat",
                   "SSR kcal",
                   "emissions",
                   "herd size"]

    def _calculate(self, x_tuple, verbosity):
        
        x = list(x_tuple)
        # Only recompute if not already cached
        
        if x_tuple not in self._cache:

            # Update the parameters with the current values
            params = self.params_default.copy()
            for i_name, name_string in enumerate(self.names_x):
                params[name_string] = x[i_name]
            
            # Perform the SSR and emissions calculation
            z_val = run_calculator(self.datablock_init, params)

            # cached dict
            zval_dict = {zn: zv for zn, zv in zip(self.z_names, z_val)}

            # Store the results in the cache
            self._cache[x_tuple] = zval_dict

        # Print out what's going on 
        if (verbosity > 1):
            for i_name, name_string in enumerate(self.names_x):
                print(f"{name_string} = {x[i_name]:.10f}; ", end="")
            for i_name, name_string in enumerate(list(self._cache[x_tuple].keys())):
                print(f"{name_string} = {self._cache[x_tuple][name_string]:.10f}; ", end="")
            print()

        return self._cache[x_tuple]

    # Define the objective function for minimization
    def objective(self, x, z_name_requested, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return self._calculate(x_tuple, verbosity)[z_name_requested]
    
    # Define the objective function for minimization
    def negative_objective(self, x, z_name_requested, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return -self._calculate(x_tuple, verbosity)[z_name_requested]

    # Define the objective function for SSR constraint
    def positive_constraint(self, x, key, threshold, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return self._calculate(x_tuple, verbosity)[key] - threshold
    
    def negative_constraint(self, x, key, threshold, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return threshold - self._calculate(x_tuple, verbosity)[key]
    

In [5]:
# Do a test calculation
z_val_baseline = run_calculator(datablock_init, params_baseline)
#print(f"SSR = {SSR_result:.8f}; GHG = {emissions_result:.8f}")

for zn, zval in zip(z_names, z_val_baseline):
    print(f"{zn} = {zval:.8f}; ", end="")
# This is my checksum for debugging: 
# SSR = 0.67366432; GHG = 94.22954995

SSR weight = 0.67366432; SSR prot = 0.73314955; SSR fat = 0.63337477; SSR kcal = 0.68543863; emissions = 94.77729174; herd size = 9151744.10648140; 

In [6]:
# Set tolerances, verbosity, and options for the minimizer
ffc_tol = 1e-6
options = {
    'disp': True,      # Show convergence messages
    'maxiter': 1000,     # Max number of iterations
    'rhobeg' : 10 # Reasonable step size (mostly they are percentages, so change by 10%)
}


In [7]:
ffc_wrapper = FFCObjectiveWithCache(names_x, datablock_init, params_baseline, verbosity=2)

# z_name_requested = "SSR weight"
z_name_requested = "herd size"
x0 = [params_baseline[n] for n in names_x]
ffc_constraints = [{'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR weight", threshold=0.6736643225, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR prot", threshold=0.7331495528, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR fat", threshold=0.6333747730, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR kcal", threshold=0.6854386315, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.negative_constraint(x, "emissions", threshold=0.0, verbosity=0)}]

result = minimize(
    lambda x: ffc_wrapper.negative_objective(x, z_name_requested),
    # lambda x: ffc_wrapper.objective(x, z_name_requested),
    x0,
    method='COBYLA',
    bounds=x_bounds,
    constraints=ffc_constraints,
    tol=ffc_tol,
    options=options
)

ruminant = 0.0000000000; pig_poultry = 0.0000000000; fish_seafood = 0.0000000000; dairy = 0.0000000000; eggs = 0.0000000000; fruit_veg = 0.0000000000; pulses = 0.0000000000; meat_alternatives = 0.0000000000; dairy_alternatives = 0.0000000000; waste = 0.0000000000; foresting_pasture = 13.1700000000; land_BECCS = 0.0000000000; land_BECCS_pasture = 0.0000000000; horticulture = 0.0000000000; pulse_production = 0.0000000000; lowland_peatland = 0.0000000000; upland_peatland = 0.0000000000; pasture_soil_carbon = 0.0000000000; arable_soil_carbon = 0.0000000000; mixed_farming = 0.0000000000; silvopasture = 0.0000000000; methane_inhibitor = 0.0000000000; stock_density = 0.0000000000; manure_management = 0.0000000000; animal_breeding = 0.0000000000; fossil_livestock = 0.0000000000; agroforestry = 0.0000000000; nitrogen = 0.0000000000; vertical_farming = 0.0000000000; fossil_arable = 0.0000000000; waste_BECCS = 0.0000000000; SSR weight = 0.6736643225; SSR prot = 0.7331495528; SSR fat = 0.633374773

In [8]:
result

 message: Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.
 success: False
  status: 4
     fun: -24540643.868740745
       x: [ 5.000e+01 -1.000e+02 ...  1.106e+01  4.001e+01]
    nfev: 1000
   maxcv: 0.005567797062870472

In [9]:
# The result is an OptimizeResult object
print("Optimization success:", result.success)
print("Message:", result.message)
print("Number of iterations:", result.nfev)
print("Optimal value of x:", result.x)
print("Minimum value of function:", result.fun)

Optimization success: False
Message: Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.
Number of iterations: 1000
Optimal value of x: [ 4.99988852e+01 -1.00002775e+02 -4.02753673e+01 -5.00053901e+01
 -3.83416453e+01 -3.01831094e+01  2.11106244e+01 -5.52059063e-03
  5.24343598e-01  6.43104276e+01  9.99443249e+00  1.38955323e+01
  1.94920672e+01  2.61770561e+00  1.21480741e+01  7.29817639e+00
  2.97442783e+00  2.39139351e+01  1.62683846e+01  2.50043832e+01
  8.65871333e-01  2.03432507e+01  4.99823175e+01  2.38473219e+01
  1.12469611e+01  5.81698248e+00  3.40613963e+00  1.07496816e+01
  1.08133625e+01  1.10633441e+01  4.00055661e+01]
Minimum value of function: -24540643.868740745


In [10]:
print("Optimized parameters: ", result['x'])      # Optimal parameters
print("Optimized value: ", result['fun'])    # Minimum value of the objective
print("Was the minimization succesfull? ", result['success'])  # Boolean indicating if it was successful

Optimized parameters:  [ 4.99988852e+01 -1.00002775e+02 -4.02753673e+01 -5.00053901e+01
 -3.83416453e+01 -3.01831094e+01  2.11106244e+01 -5.52059063e-03
  5.24343598e-01  6.43104276e+01  9.99443249e+00  1.38955323e+01
  1.94920672e+01  2.61770561e+00  1.21480741e+01  7.29817639e+00
  2.97442783e+00  2.39139351e+01  1.62683846e+01  2.50043832e+01
  8.65871333e-01  2.03432507e+01  4.99823175e+01  2.38473219e+01
  1.12469611e+01  5.81698248e+00  3.40613963e+00  1.07496816e+01
  1.08133625e+01  1.10633441e+01  4.00055661e+01]
Optimized value:  -24540643.868740745
Was the minimization succesfull?  False


In [11]:
# Display the results
#z1_val, z2_val = list(ffc_results.values())[:2]
z1_val = ffc_wrapper.objective(result.x, "SSR weight")
z2_val = ffc_wrapper.objective(result.x, "emissions")
print(f"SSR weight = {z1_val:.8f}; emissions = {z2_val:.8f}")


ruminant = 49.9988852077; pig_poultry = -100.0027753392; fish_seafood = -40.2753673056; dairy = -50.0053901492; eggs = -38.3416453200; fruit_veg = -30.1831094322; pulses = 21.1106244293; meat_alternatives = -0.0055205906; dairy_alternatives = 0.5243435980; waste = 64.3104276475; foresting_pasture = 9.9944324916; land_BECCS = 13.8955323128; land_BECCS_pasture = 19.4920671723; horticulture = 2.6177056132; pulse_production = 12.1480740687; lowland_peatland = 7.2981763870; upland_peatland = 2.9744278257; pasture_soil_carbon = 23.9139351452; arable_soil_carbon = 16.2683845648; mixed_farming = 25.0043832088; silvopasture = 0.8658713331; methane_inhibitor = 20.3432507050; stock_density = 49.9823174865; manure_management = 23.8473219444; animal_breeding = 11.2469610631; fossil_livestock = 5.8169824808; agroforestry = 3.4061396297; nitrogen = 10.7496816175; vertical_farming = 10.8133625107; fossil_arable = 11.0633441288; waste_BECCS = 40.0055661048; SSR weight = 0.7476475335; SSR prot = 0.77830